In [18]:
from keras.datasets import mnist
import matplotlib
# %matplotlib inline
import matplotlib.pyplot as plt
import l2_attack
import keras
from defense import *

In [15]:
xin = tf.placeholder(tf.float32, [30, 128])

session = keras.backend.get_session()
mygan = Generator(30, xin)

keras.backend.set_learning_phase(False)
model = keras.models.load_model("data/mnist")

touse = [x for x in tf.trainable_variables() if 'Generator' in x.name]
saver = tf.train.Saver(touse)
saver.restore(session, 'data/mnist-gan')

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = np.array(x_test, dtype=np.float32)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test /= 255.0

# 1. solve the inversion problem                                                                                                                                                               
attack1 = l2_attack.CarliniL2(session, model,
                              lambda x: Generator(30, x),
                              binary_search_steps=1,
                              max_iterations=3000,
                              learning_rate=1e-1,
                              batch_size=30,
                              initial_const=0, targeted=None)

# 2. start there, make it adversarial                                                                                                                                                          
attack2 = l2_attack.CarliniL2(session, model,
                              lambda x: Generator(1, x),
                              binary_search_steps=5,
                              max_iterations=30000,
                              learning_rate=1e-1,
                              batch_size=1,
                              initial_const=1, targeted=False,
                              noise=False, abort_early=False)



In [16]:
print("True label", y_test[0])
print("Preds",model.predict(x_test)[0])
plt.imshow(x_test[0,:,:,0])
plt.show()

res = attack1.attack([x_test[0]]*30,
                     [np.eye(10)[0]]*30)

start = session.run(attack1.modifier)
it = session.run(mygan, {xin: start})
distortion = np.sum((it-[x_test[0]]*30)**2,(1,2,3))**.5
print("Distortions", distortion)
start = np.array([start[np.argmin(distortion)]])

res = session.run(Generator(1,
                            tf.constant(start, dtype=tf.float32)))
print(np.sum((res-x_test[:1])**2,(1,2,3))**.5)
print(np.mean(np.sum((res-x_test[:1])**2,(1,2,3))**.5))

res = attack2.attack(x_test[:1],
                    [np.eye(10)[q] for q in y_test[:1]],
                    start)

print("L2 Distortion", np.sum((res-x_test[:1])**2)**.5)

print("Preds",model.predict(res)[0])
plt.imshow(res[0,:,:,0])
plt.show()

True label 7
Preds [1.6938644e-10 1.3955457e-09 2.7626207e-08 6.7455002e-09 1.8495329e-11
 2.6476349e-12 3.8272548e-15 1.0000000e+00 1.4897891e-10 3.1801573e-08]
go up to 30
tick 0


W0219 17:08:32.488272 139808513279808 module_wrapper.py:139] From /home/zhouyi/obfuscated-gradients/defensegan/l2_attack.py:178: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



0 (2454.3506, 0.0, 2454.3506)
300 (803.59515, 0.0, 803.59515)
600 (709.4947, 0.0, 709.4947)
900 (676.4509, 0.0, 676.4509)
1200 (601.7418, 0.0, 601.7418)
1500 (577.74744, 0.0, 577.74744)
1800 (568.8697, 0.0, 568.8697)
2100 (562.7727, 0.0, 562.7727)
2400 (562.6991, 0.0, 562.6991)
2700 (489.63208, 0.0, 489.63208)
Distortions [7.0524673  6.3200884  0.48555344 0.5305     6.4789357  0.48962137
 3.394742   6.886721   0.4675297  0.63765526 5.068954   6.808161
 2.3322198  1.0930243  0.48068076 2.0573401  6.943852   7.189198
 0.44810373 2.818974   0.6638034  0.5538179  4.1424522  0.52029365
 1.7132046  0.48089895 6.961357   0.5815713  0.49459863 0.68951845]
[0.44810358]
0.44810358
go up to 1
tick 0
0 (35.56478, 14.217804, 21.34698)
3000 (8.924093, 2.940808, 5.983285)
6000 (7.897152, 2.0997279, 5.797424)
9000 (7.146699, 1.4372644, 5.7094345)
12000 (7.0951986, 1.2353106, 5.859888)
15000 (6.944693, 0.9200654, 6.0246277)
18000 (6.8542223, 0.8006993, 6.053523)
21000 (6.8981433, 0.7902493, 6.107894)
2

In [19]:
plt.imshow(res[0,:,:,0])